# Importing Packages

In [1]:
# Basic
import pandas as pd
import json
import collections
from collections import defaultdict
import itertools
from itertools import chain
import urllib
from urllib.parse import urlparse
import re
import numpy as np
import datetime
import mysql.connector

# Advanced
from pandas.io.json import json_normalize
import operator
from operator import itemgetter

# Language processing
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet as wn
from gensim.models import word2vec
from gensim import corpora
from gensim.models import CoherenceModel
from gensim.models.keyedvectors import KeyedVectors
from wordsegment import load, segment
load() # for spell checking
SpellCheck = segment

# Visualizations
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

# Establishing Connection

In [2]:
config_own = {
        'user' : 'root', 
        'password' :'MacBookAir1231', 
        'host' : '127.0.0.1',
        'database' : 'TargetDB',
        'ssl_disabled' : 'True'
    }
con_own = mysql.connector.connect(**config_own)

DatabaseError: 2003 (HY000): Can't connect to MySQL server on '127.0.0.1' (61)

# Query Loader

In [ ]:
test_own = pd.read_sql_query('select entry from requestlog limit 20000', con_own)

In [4]:
test_own.head(10)

,entry
0,"{""visitorId"":""63077af1-c15d-40e8-9b2b-64c36d18..."
1,"{""visitorId"":""0f4c7121-f5ca-42ef-aad4-e6ed09af..."
2,"{""visitorId"":""7f0ac3e8-0a1e-4ae3-88c2-5daa045c..."
3,"{""visitorId"":""318666e5-b268-4bb7-87c3-04141445..."
4,"{""visitorId"":""a66172dc-85b7-4814-97c1-65ce52a0..."
5,"{""visitorId"":""2b00c776-8e5b-4d2c-b72d-84ac31e3..."
6,"{""visitorId"":""318666e5-b268-4bb7-87c3-04141445..."
7,"{""visitorId"":""318666e5-b268-4bb7-87c3-04141445..."
8,"{""visitorId"":""318666e5-b268-4bb7-87c3-04141445..."
9,"{""visitorId"":""318666e5-b268-4bb7-87c3-04141445..."


# Making Data Dump

In [4]:
query = 'select entry from requestlog limit 1000'
path = 'query_sample_paper'

In [6]:
def query_to_json_file(query, column, path_file):
        index = 0
        data_json = []

        while (index < len(query)):
            j = query.get_value(index, column)
            #my_json = j.decode('utf8')
            data = json.loads(j)
            data_json.append(data)
            index += 1

        print("DONE 1")

        if path_file != "":
            with open(path_file, 'w') as outfile:
                json.dump(data_json, outfile, indent=0)

            print("DONE 2")

        else:
            return data_json

In [7]:
query_to_json_file(test_own,'entry',path)

/Users/Dimitar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


DONE 1
DONE 2


# Testing Json Dump

In [3]:
def json_read(filepath, multiline=False):
        file = pd.read_json(filepath, lines=multiline, convert_dates=False)
        print("Step 1/7 - Reading, done...")
        return file

In [5]:
path = 'query_sample'
json_test = json_read(path)

Step 1/7 - Reading, done...


# Filtering

In [7]:
def filter_columns(data_frame):
        collector_data = json_normalize(data_frame['collectorData'])
        all_data = pd.concat([collector_data, data_frame], axis=1)
        keep_list = ['visitorId', 'timestamp', 'pageUrl',
                     'newVisit', 'pageId']
        processed_data = all_data[keep_list]
        print("Step 2/7 - Filtering, done...")
        return processed_data

In [8]:
filter_test = filter_columns(json_test)

Step 2/7 - Filtering, done...


In [9]:
# fixing unix timestamp
def formatTime(value):
    timestamp, ms = divmod(value, 1000)
    dt = datetime.datetime.fromtimestamp(timestamp) + datetime.timedelta(milliseconds=ms)
    formatted_time = dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    return formatted_time

In [13]:
formatTime(1555299316393)

'2019-04-15 05:35:16.393'

In [15]:
formatTime(1472489163111)

'2016-08-29 18:46:03.111'

In [10]:
filter_test['timestamp'] = filter_test.apply(lambda row: formatTime(row['timestamp']), axis=1)

In [11]:
filter_test.head(5)

,visitorId,timestamp,pageUrl,newVisit,pageId
0,63077af1-c15d-40e8-9b2b-64c36d188658,2016-08-29 18:46:05.809,https://www.onehippo.org/library/administratio...,True,hst:pages/documentation
1,0f4c7121-f5ca-42ef-aad4-e6ed09afcdde,2016-08-29 18:46:03.111,https://www.onehippo.com/en/digital-experience...,True,hst:pages/Digital-experience-platform
2,7f0ac3e8-0a1e-4ae3-88c2-5daa045c5500,2016-08-29 18:46:09.518,https://www.onehippo.org/,True,hst:pages/home
3,318666e5-b268-4bb7-87c3-04141445a061,2016-08-29 18:46:11.279,https://www.onehippo.org/,True,hst:pages/home
4,a66172dc-85b7-4814-97c1-65ce52a02b21,2016-08-29 18:46:14.663,http://www.onehippo.com/connect/boston,True,hst:pages/boston


# Sorting

In [12]:
def json_sort(file):
        sort_by = ["visitorId", "timestamp"]
        sorted_file = file.sort_values(by=sort_by)
        print("Step 3/7 - Sorting, done...")
        return sorted_file

In [13]:
sortedData=json_sort(filter_test)

Step 3/7 - Sorting, done...


# Temporary - testing to go withthout transaction

In [15]:
def remove_homepage(data_frame):
        data_frame = data_frame.drop(
            data_frame[(
                        (data_frame.pageId == 'hst:pages/home') |
                        (data_frame.pageId == 'hst:pages/pagenotfound')
                        | (data_frame.pageId == 'hst:pages/robotstxt'))
                       ].index).reset_index(drop=True)
        print("Step 6/7 - Remove visitors that only visited the homepage, done...")
        return data_frame

In [16]:
sortedData = remove_homepage(sortedData)
#sortedData = clean_homepage[clean_homepage.astype(str)['keywords'] != '[]']

Step 6/7 - Remove visitors that only visited the homepage, done...


In [21]:
def get_content_page_and_keywords(data_frame):
        data_frame['keywords'] = data_frame['pageUrl'].map(lambda x: checkForList(x))
        #data_frame['contentPage'] = data_frame.transactionPath.str[-1]
        print("Step 5/7 - Keep content pages and get path keywords, done...")
        return data_frame

In [22]:
sortedData = get_content_page_and_keywords(sortedData)

Step 5/7 - Keep content pages and get path keywords, done...


In [23]:
len(sortedData)

15264

In [24]:
sortedData = sortedData.groupby('visitorId', as_index=False).agg(lambda x: x.tolist())

In [25]:
sortedData.head(5)

,visitorId,timestamp,pageUrl,newVisit,pageId,keywords
0,0017bf12-aece-4c5d-a0df-eb6e93e10030,"[2016-08-30 14:11:50.959, 2016-08-31 12:05:13....","[https://www.onehippo.com/de/warum-hippo-cms, ...","[True, True]","[hst:pages/why-hippo-cms-default, hst:pages/bl...","[[cms, hippo, warum], [resource, versioning, b..."
1,00220802-a7ff-4638-9fa0-05d872dd7474,[2016-08-31 06:23:58.689],[https://www.onehippo.com/en/careers],[True],[hst:pages/careers],[[career]]
2,00312144-f7b4-4eb4-9eef-038b353d7382,"[2016-08-30 00:59:55.358, 2016-08-30 01:08:12....",[https://www.onehippo.com/en/customers/nationa...,"[True, False, False]","[hst:pages/customerdetail, hst:pages/customerd...","[[police, national, customer], [police, nation..."
3,00428e69-52e7-4759-a645-c14aac5f0bbe,[2016-08-31 16:55:16.461],[http://onehippo-prod.global.ssl.fastly.net/si...,[True],[hst:components/redirect],"[[dnt, policy, well, know, site]]"
4,005c41f9-ab4e-410c-99b6-381b37f8209d,[2016-08-29 19:48:15.164],[https://www.onehippo.org/library/concepts/plu...,[True],[hst:pages/documentation],"[[plugins, concept, tag, library, configuration]]"


In [26]:
sortedData=sortedData.drop(['timestamp', 'pageId','newVisit'], axis=1)

In [27]:
def unlistMeDF(listObject):
    flat_list = []
    for sublist in listObject:
        for item in sublist:
            flat_list.append(item)
    return flat_list

In [28]:
sortedData['keywords'] = sortedData['keywords'].apply(lambda x: unlistMeDF(x))

In [29]:
len(sortedData)

5498

In [30]:
sortedData.head(5)

,visitorId,pageUrl,keywords
0,0017bf12-aece-4c5d-a0df-eb6e93e10030,"[https://www.onehippo.com/de/warum-hippo-cms, ...","[cms, hippo, warum, resource, versioning, blog..."
1,00220802-a7ff-4638-9fa0-05d872dd7474,[https://www.onehippo.com/en/careers],[career]
2,00312144-f7b4-4eb4-9eef-038b353d7382,[https://www.onehippo.com/en/customers/nationa...,"[police, national, customer, police, national,..."
3,00428e69-52e7-4759-a645-c14aac5f0bbe,[http://onehippo-prod.global.ssl.fastly.net/si...,"[dnt, policy, well, know, site]"
4,005c41f9-ab4e-410c-99b6-381b37f8209d,[https://www.onehippo.org/library/concepts/plu...,"[plugins, concept, tag, library, configuration]"


In [31]:
# Make for data Model
keywords_list = sortedData["keywords"].tolist()
data_NN = []
for i in keywords_list:
    data_NN.append(' '.join(i))
with open('data_NN.txt', 'w') as f:
    for item in data_NN:
        f.write("%s\n" % item)

# Get Transaction

In [14]:
## Go over it
class MFAlgorithm:

    @staticmethod
    def run_MF_algorithm(visitor, time, urls):
        url_pairs = [('', urls[0], 0, 0)]
        i = 0
        number_of_urls = len(urls)
        while (i + 1) < number_of_urls:
            url_pairs.append((urls[i], urls[i + 1], i, i + 1))
            i += 1

        i = 0
        end_transaction = False
        all_transactions = []
        current_transaction = []
        timestamp = time[0]
        number_of_pairs = len(url_pairs)

        while i < number_of_pairs:
            current_url, next_url, index_current, index_next = url_pairs[i]

            # Initialize the transaction for the first URL
            if current_url == '':
                current_transaction.append(next_url)
                timestamp = time[index_next]
                i += 1
                continue

            # If the URL exists in the transaction, end transaction and add it to list.
            # If not, we add the url to the transaction list and go on.
            if next_url in current_transaction:
                if not end_transaction:
                    if current_transaction not in all_transactions:
                        all_transactions.append((visitor, timestamp, current_transaction))
                this_index = current_transaction.index(next_url)
                current_transaction = current_transaction[0:this_index + 1]
                end_transaction = True
                i += 1
                continue
            else:
                if end_transaction:
                    end_transaction = False
                    timestamp = time[index_current]
                current_transaction.append(next_url)

            i += 1

        if current_transaction not in all_transactions:
            all_transactions.append((visitor, timestamp, current_transaction))
        return all_transactions

    @staticmethod
    def init_algorithm(sortedData):
        result = []
        grouped = sortedData.groupby('visitorId')
        i = 0
        visitor_length = len(grouped)
        print("Initializing Transaction Extraction...")
        for visitorId, group in grouped:
            time = grouped.get_group(visitorId).timestamp.tolist()
            path = grouped.get_group(visitorId).pageUrl.tolist()
            result_paths = MFAlgorithm.run_MF_algorithm(visitorId, time, path)
            result.extend(result_paths)
            i += 1
            if i % 1000 == 0:
                print("Progress:", round((i / visitor_length) * 100, 2), "%")
        return result

In [15]:
mfa = MFAlgorithm()
def get_transactions(sorted_data):
        transactions = mfa.init_algorithm(sorted_data)
        data_frame = pd.DataFrame(transactions, columns=['visitorId', 'timestamp', 'transactionPath'])
        data_frame = pd.merge(data_frame, sorted_data, on=['visitorId', 'timestamp'])
        print("Step 4/7 - Extract transactions, done...")
        return data_frame.drop(['timestamp', 'pageUrl'], axis=1)

In [16]:
fixed_transaction = get_transactions(sortedData)

Initializing Transaction Extraction...
Progress: 12.44 %
Progress: 24.88 %
Progress: 37.31 %
Progress: 49.75 %
Progress: 62.19 %
Progress: 74.63 %
Progress: 87.06 %
Progress: 99.5 %
Step 4/7 - Extract transactions, done...


# Get Keywords

In [17]:
def morphForms(word):
    if word == 'cms':
        return word
    examined_word = wn.morphy(word)
    if examined_word is None:
        return word
    else:
        return examined_word
    
# Example: discouraged(word) discourage (returned function) or cats(word) returned cat

In [18]:
# Not using at the moment as spell checking might be tricky
# Example: statististics (initial word), and is split into ['statist', 'istics']

def fixConcatWords(listd):
    checkSpelling = []
    second = []
    for a in listd:
        second.append(a)
        checkSpelling.append(SpellCheck(a))
    value = [x for x in checkSpelling if x not in second]
    unlisted = [item for sublist in value for item in sublist]
    '''
    # This part is adding the concatinated word to the list, but i do not need it, maybe use later
    for item in itertools.chain(unlisted, second):
        if item not in unlisted:
            unlisted.append(item)
            '''
    return unlisted

In [44]:
def get_keywords(d):
    lemmatizer = WordNetLemmatizer()
    sw = set(stopwords.words('english'))
    badlinks=['search.yahoo','yandex', 'google']
    dashed = []
    dups = []
    split_one = urlparse(d)
    split_two = split_one._asdict()
    for k,v in split_two.items():
        if k == 'path':
            # Basic:
            value = v.replace(".html", "")
            value = value.replace(".xml", "")
            value = value.replace(".php", "")
            # Advanced: 
            value = re.sub("[\d+ % &]", " ", value) # removes numbers and replaces with empty space
            value = re.sub("(?<=[' '])[.,-]|[.,-](?=[' '])", " ", value) # remove dashes when there is empty space infront or behind
            value = re.sub('(?<=\D)[.,_]|[.,_](?=\D)', ' ', value) # removes dashes, dots, commas if there is a digit before/after
            #value = re.sub('[^A-Za-z0-9]+', '', value) # removes all dots,dashed etc
            value = value.replace(' ','-')
            value = re.sub('\s\s+', ' ',value) # remove double empty spaces
            #value = re.sub('\s+', '',value) # remove empty spaces
            value = value.split('/')
            value = list(filter(None, value))
            sw.add('en')
            sw.add('nl')
            sw.add('de')
            sw.add('txt')
            sw.add('oh')
            sw.add('net')
            sw.add('com')
            sw.add('1')
            sw.add('login')
            sw.remove('about')
            sw.remove('why')
            sw.remove('in')
            
            # manually splitting words with _ and -
            for a in value:
                if '_' in a:
                    b = a.replace('_','-')
                    dashed.append(b.split('-'))
                    dups.append(a)
                if '-' in a:
                    dashed.append(a.split('-'))
                    dups.append(a)
    
            value = [x for x in value if x not in dups]
            resultList=set(value)
            value_dash = set(list(chain.from_iterable(dashed))) # merge and keep unique ones
            resultList.update(value_dash)
            #resultList = fixConcatWords(resultList) # not used atm, check function for notes
            resultList = [w for w in resultList if w not in sw]
            resultList = [x.lower() for x in resultList]
            resultList = [morphForms(word) for word in resultList]
            resultList = list(filter(None, resultList)) # fastest
            resultList = set(resultList)
            # new
            final_list = []
            for i in resultList:
                word_tokens = nltk.word_tokenize(i) ###Tokenization
                stemmed_words = [lemmatizer.lemmatize(w.lower()) for w in word_tokens if w.isalpha()] ###Stemming., removes some git, example line 7
                final_list.append(stemmed_words)
            # new until here
            #resultList = list(resultList)
            final_list =[item for sublist in final_list for item in sublist]
            
    return final_list

In [ ]:
# Usefull for spelling correction, but doesnt make sense for me
'''
import os
from symspellpy.symspellpy import SymSpell  # import the module
# https://github.com/mammothb/symspellpy Source

# maximum edit distance per dictionary precalculation
max_edit_distance_dictionary = 0
prefix_length = 7
# create object
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
# load dictionary
dictionary_path = os.path.join("frequency_dictionary_en_82_765.txt")
term_index = 0  # column of the term in the dictionary text file
count_index = 1  # column of the term frequency in the dictionary text file
if not sym_spell.load_dictionary(dictionary_path, term_index, count_index):
    print("Dictionary file not found")

# a sentence without any spaces
input_term = "thequickbrownfoxjumpsoverthelazydog"
    
result = sym_spell.word_segmentation(input_term)
# display suggestion term, term frequency, and edit distance
print("{}".format(result.corrected_string))

# Output: the quick brown fox jumps over the lazy dog
'''

In [45]:
def checkForList(listd):
    tempList = []
    if isinstance(listd,(list,)):
        for i in listd:
            tempList.append(get_keywords(i))
    else:
        return get_keywords(listd)
    
    flat_list = [item for sublist in tempList for item in sublist]
    return flat_list


In [46]:
def get_content_page_and_keywords(data_frame):
        data_frame['keywords'] = data_frame['transactionPath'].map(lambda x: checkForList(x))
        data_frame['contentPage'] = data_frame.transactionPath.str[-1]
        print("Step 5/7 - Keep content pages and get path keywords, done...")
        return data_frame

In [47]:
keywords_df = get_content_page_and_keywords(fixed_transaction)

Step 5/7 - Keep content pages and get path keywords, done...


# Remove homepage

In [48]:
def remove_homepage(data_frame):
        data_frame = data_frame.drop(
            data_frame[(
                        (data_frame.pageId == 'hst:pages/home') |
                        (data_frame.pageId == 'hst:pages/pagenotfound')
                       )
                       &
                       (data_frame.transactionPath.str.len() == 1)
                       ].index).reset_index(drop=True)
        print("Step 6/7 - Remove visitors that only visited the homepage, done...")
        return data_frame

In [49]:
clean_homepage = remove_homepage(keywords_df)
clean_homepage = clean_homepage[clean_homepage.astype(str)['keywords'] != '[]']

Step 6/7 - Remove visitors that only visited the homepage, done...


In [50]:
clean_homepage.head(10)

,visitorId,transactionPath,newVisit,pageId,keywords,contentPage
0,0017bf12-aece-4c5d-a0df-eb6e93e10030,"[https://www.onehippo.com/de/warum-hippo-cms, ...",True,hst:pages/why-hippo-cms-default,"[cm, warum, hippo, resource, semantic, version...",https://www.onehippo.com/en/resources/blogs/20...
1,00220802-a7ff-4638-9fa0-05d872dd7474,[https://www.onehippo.com/en/careers],True,hst:pages/careers,[career],https://www.onehippo.com/en/careers
2,00312144-f7b4-4eb4-9eef-038b353d7382,[https://www.onehippo.com/en/customers/nationa...,True,hst:pages/customerdetail,"[police, national, customer]",https://www.onehippo.com/en/customers/national...
3,00312144-f7b4-4eb4-9eef-038b353d7382,[https://www.onehippo.com/en/customers/nationa...,False,hst:pages/customerdetail,"[police, national, customer, partner, undefined]",https://www.onehippo.com/en/undefined
4,00428e69-52e7-4759-a645-c14aac5f0bbe,[http://onehippo-prod.global.ssl.fastly.net/si...,True,hst:components/redirect,"[dnt, site, well, policy, know]",http://onehippo-prod.global.ssl.fastly.net/sit...
5,005c41f9-ab4e-410c-99b6-381b37f8209d,[https://www.onehippo.org/library/concepts/plu...,True,hst:pages/documentation,"[plugins, tag, configuration, library, concept]",https://www.onehippo.org/library/concepts/plug...
6,005dbc0e-b0a6-461c-bdad-4d775bdcae30,"[https://www.onehippo.com/robots.txt, http://w...",True,hst:pages/robotstxt,"[robot, connect, boston]",http://www.onehippo.com/connect/boston
7,005dbc0e-b0a6-461c-bdad-4d775bdcae30,"[https://www.onehippo.com/robots.txt, https://...",False,hst:pages/robotstxt,"[robot, cloud, enterprise, financial, service,...",https://www.onehippo.com/en/open-source
8,005dbc0e-b0a6-461c-bdad-4d775bdcae30,"[https://www.onehippo.com/robots.txt, https://...",False,hst:pages/pagenotfound,"[robot, cloud, enterprise, financial, service,...",https://www.onehippo.com/en/why-hippo-cms
9,0061ed93-f80e-440e-b054-40f9cd4287c3,[http://www.onehippo.com/connect/amsterdam],True,hst:pages/amsterdam,"[connect, amsterdam]",http://www.onehippo.com/connect/amsterdam


# Making Data For NeuralNetwork and Scrapping

In [27]:
# Make for data Model
keywords_list = clean_homepage["keywords"].tolist()
data_NN = []
for i in keywords_list:
    data_NN.append(' '.join(i))
with open('data_NN.txt', 'w') as f:
    for item in data_NN:
        f.write("%s\n" % item)

In [ ]:
# Prepare links for Scrapping
url_list = clean_homepage["contentPage"].tolist()
url_file = []

for i in url_list:
    url_file.append(i)
    
url_file = set(url_file) # Cleaning duplicates
url_file = list(url_file)

with open('url_file.txt', 'w') as f:
    for item in url_file:
        f.write("%s\n" % item)

# Testing - TopicModels

In [ ]:
'''
    The idea of this section is to make use of different Topic Modelling techniques in order to do basic and straighforward user
    segmentation based on their keywords.
    Using custom dictionary (data) and the distance function we can assign the most suitable category based on distance
'''

In [ ]:
def build_corpus(data):
    "Creates a list of lists containing words from each sentence"
    corpus = []
    temp = []
    for sentence in data['keywords'].iteritems():
        #print(sentence)
            #print(blq1)
            #word_list = sentence[1].split(" ")
        corpus.append(sentence[1])
            
    return corpus

corpus = build_corpus(clean_homepage)        
corpus[0:2]

In [ ]:
pd.options.mode.chained_assignment = None 
dictionary = corpora.Dictionary(corpus)
corpus = [dictionary.doc2bow(text) for text in corpus]

In [ ]:
dictionary[100]

In [ ]:
corpus[100]

In [ ]:
'''
# Below is another approach for gathering topics 

# The Mallet LDA, it gives way tooo many topics

# https://stackoverflow.com/questions/50340657/pyldavis-with-mallet-lda-implementation-ldamallet-object-has-no-attribute-inf
# to use the the visualization bellow
model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=8, id2word=dictionary)
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(model)

topics_Mallet = model.print_topics(num_words=4)
for topic in topics_Mallet:
    print(topic)
'''

In [ ]:
import gensim, warnings, logging
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

NUM_TOPICS = 5      
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=250, 
                                           per_word_topics=True, alpha='auto')

#ldamodel.save('model2604.gensim') # save model

In [ ]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
# Compute Perplexity
print ('Perplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
lemmatized = build_corpus(clean_homepage) 

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=lemmatized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print ('Coherence Score: ', coherence_lda)

In [ ]:
import pyLDAvis.gensim, warnings, logging
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary=dictionary)
vis

In [ ]:
# the lenght of data lists should be equal because of the distance function
# Category -> words
data = {
  'RFC/Support': ['application','component','concept','content','editor','handling','library','plugins','repository','request','support'], #0 # bubble 1

  'Tool/Application': ['cms','developer','download','extensibility','getting','hippo','scalability','start','trail','user','why'], #1 # bubble 2

  'Reporting/Consulting': ['career','consult','event','fundamentals','generate','operations','report','sales','training'],#2 #bubble 5

  'Deployment': ['angular','boston','building','cloud','connect','deploy','deployment','linux','management','prepare','resource','running','system'], #3 # bubble 3

  'Customer/Product Related': ['agile','customer','demo','digital','experience','industry','partner','platform','product','technology'] #4 #buble 4 
}
# Dict: Words -> Category
categories = {word: key for key, words in data.items() for word in words}

# Load the whole embedding matrix
embeddings_index = {}
with open('glove.6B.300d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        embed = np.array(values[1:], dtype=np.float32)
        embeddings_index[word] = embed
print('Loaded {} word vectors.'.format(len(embeddings_index))

# Get embeddings
data_embeddings = {key: value for key, value in embeddings_index.items() if key in categories.keys()}

# Processing the query
# Might take some time in case 'data' is bigger
def process(query):
    query_embed = embeddings_index[query] # get wordEmbedding of query
    scores = {}
    for word, embed in data_embeddings.items():
        category = categories[word]
        distance = query_embed.dot(embed)
        distance /= len(data[category]) # divide by the len of the dict list
        scores[category] = scores.get(category, 0) + distance # If category is there add to it if not add to 0
    return scores


In [ ]:
b = process('process')
def getMaxValue(d):
    return max(d.items(), key = operator.itemgetter(1))[0]
getMaxValue(b)

# Test return of highest category
a = ['library', 'concepts', 'workflow', 'workflow']
topics = []
for i in a:
    temp = process(i)
    topics.append(getMaxValue(temp))
print(topics)

In [ ]:
def FillInCateg(d):
    topics = []
    for i in d:
        try:
            temp = process(i)
            topics.append(getMaxValue(temp))
        except:
            continue
    topics = set(topics)
    topics = list(topics)
    return topics

In [ ]:
def FillInCategMultiple(d):
    '''
        Logic is as follows:
        - We create empty list topics, which will store and return the final results
        - Two empty dictionries are created - tempComb, which will temporary store the first result and will then transfer it to combined
        - then we check if the length of the input is exactly 1 (single item list), then we just return the highest value
            - if this is not the case we go to the for loop:
                - try to get the categ dictionary (maybe the word has no pretrained vector)
                - have temp variable - check, which is initiated as 0, at first run it will help us copy the first dictionary to tempComb
                - on the second run, the first dictionary is updated and then we go in the in else part and we basically update the tempComb
                value per key
                - once this is done, we create a new dict - final, which is sorted based on the values
                    - we take first and second value and check their difference
                        - if difference is bigger than 1, we just take first item
                        - if diff is smaller we take the first two
    '''
    topics = []
    tempComb = {}
    check = 0
    if len(d) == 1:
        singleValue, = d
        topics.append(getMaxValue(process(singleValue)))
        topics = set(topics)
        topics = list(topics)
        return topics
    for i in d:
        try:
            perwordCateg = process(i)
            if check == 0:
                tempComb = perwordCateg
                check = 1
            else:
                for (k,v), (k2,v2) in zip(tempComb.items(), perwordCateg.items()):
                    if k == k2:
                        tempComb[k] += v2
        except:
            continue

    final = sorted(tempComb.items(), key=itemgetter(1), reverse=True)
    try:
        first = final[0][1]
        second = final[1][1]

        if (first - second) > 1:
            topics.append(final[0][0])
        else:
            topics.append(final[0][0])
            topics.append(final[1][0])
        
        topics = set(topics)
        topics = list(topics)
    except Exception as e:
        print('List {}, gave the following error: {}'.format(d,e))
        issue = 'Issue ' + str(d)
        return issue
    return topics

In [ ]:
clean_homepage['category_perWord'] = clean_homepage['keywords'].map(lambda x: FillInCateg(x))

In [ ]:
clean_homepage['category_combined'] = clean_homepage['keywords'].map(lambda x: FillInCategMultiple(x))

In [ ]:
clean_homepage

In [ ]:
# Testing
print(process('programming'))
print(process('how'))
print(process('instructions'))
print(process('customer'))
print(process('present'))
print(process('presentation'))
print(process('design'))
print(process('omni'))
print(process('channel'))

In [ ]:
# Load word2vec gensim
w2v = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

In [ ]:
def getFirstTuple(d):
    result = []
    temp = w2v.similar_by_vector(d,3)
    for a in temp:
        result.append(a[0])
    return result

In [ ]:
def getMean(d):
    values = np.zeros(300)
    end = np.zeros(300)
    for i in d:
        k = 0 
        if i in w2v:
            values += w2v[i]
            k += 1
        end = values/k
    return end

In [ ]:
clean_homepage['AvgVectorKeywords'] = clean_homepage['keywords'].map(lambda x: getFirstTuple(getMean(x)))

In [ ]:
clean_homepage

In [ ]:
# Export
file_name = 'TrainingDataCreation3004.csv'
clean_homepage.to_csv(file_name, sep='\t', encoding='utf-8')

In [ ]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

# Word2Vec with our own data - Test

In [ ]:
# training Word2Vec with our data
# Pretty bad as we do not have that much
corpusWord2Vec = build_corpus(clean_homepage)  
model = word2vec.Word2Vec(corpusWord2Vec, size=100, window=20, min_count=1, workers=4)
for word in model.wv.vocab:
    print(word)

In [ ]:
tsne_plot(model)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()
 
X = []
for word in model.wv.vocab:
    X.append(model.wv[word])

X = np.array(X)
print("Computed X: ", X.shape)
#X_embedded = TSNE(n_components=2,init='pca', n_iter=2500, random_state=23,verbose=2).fit_transform(X)
X_embedded = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23).fit_transform(X)
print("Computed t-SNE", X_embedded.shape)
 
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = X_embedded[:,0], X_embedded[:,1], model.wv.vocab
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [ ]:
model.most_similar('project')